In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
# Import libraries
import pandas as pd
import os
from pathlib import Path
from tqdm import tqdm
import yaml
import matplotlib.pyplot as plt
from ultralytics import YOLO
import numpy as np
from PIL import Image, ExifTags
import torch

In [3]:
# INPUT_DIRS
INPUT_DATA_DIR = Path('dataset')
## Drop the Folder if it already exists
DATASETS_DIR = Path('dataset')
# Image & labels directory
TRAIN_IMAGES_DIR = DATASETS_DIR / 'images' / 'train'
TRAIN_LABELS_DIR = DATASETS_DIR / 'labels'/ 'train'
TEST_IMAGES_DIR = DATASETS_DIR / 'images' / 'test'
VAL_IMAGES_DIR = DATASETS_DIR / 'images' /'val'
VAL_LABELS_DIR = DATASETS_DIR / 'labels' /'val'

# Load train and test files
train = pd.read_csv(INPUT_DATA_DIR / 'Train_df.csv')
val = pd.read_csv(INPUT_DATA_DIR / 'Val_df.csv')
test = pd.read_csv(INPUT_DATA_DIR / 'Test.csv')
ss = pd.read_csv(INPUT_DATA_DIR / 'SampleSubmission.csv')

class_map = {cls: i for i, cls in enumerate(sorted(train['class'].unique().tolist()))}
# Strip any spacing from the class item and make sure that it is a str
train['class'] = train['class'].str.strip()

# Map {'healthy': 2, 'cssvd': 1, anthracnose: 0}
train['class_id'] = train['class'].map(class_map)

train_df = train
val_df = val

# Create a data.yaml file required by yolo
class_names = sorted(train['class'].unique().tolist())
num_classes = len(class_names)
data_yaml = {
	"path" : str(DATASETS_DIR.absolute()),
	'train': str(TRAIN_IMAGES_DIR.absolute()),
	'val': str(VAL_IMAGES_DIR.absolute()),
	'test': str(TEST_IMAGES_DIR.absolute()),
	'nc': num_classes,
	'names': class_names
}

val_image_names = [str(Path(name).stem) for name in val_df['Image_ID'].unique()]
train_image_names = [str(Path(name).stem) for name in train['ImagePath'].unique()]

In [4]:
val_df['Image_ID'].nunique()

1106

In [5]:
from glob import glob

# Validate the model on the validation set
BEST_PATH = sorted(glob("runs/detect/train*/weights/best.pt"))[-1]
BEST_PATH

'runs/detect/train5/weights/best.pt'

In [6]:
# Load the trained YOLO model
model = YOLO(BEST_PATH)

model = model.eval()

In [7]:
from ultralytics.engine.predictor import BasePredictor

BEST_CONFIG = sorted(glob("runs/detect/train*/args.yaml"))[-1]
predictor = BasePredictor(BEST_CONFIG)

In [8]:
for flag, v in ExifTags.TAGS.items():
    if v == "Orientation":
        break


def load_image(filepath):
    image = Image.open(filepath)

    exif = image._getexif()
    if exif is None:
        return image

    orientation_value = exif.get(flag, None)

    if orientation_value == 3:
        image = image.rotate(180, expand=True)
    elif orientation_value == 6:
        image = image.rotate(270, expand=True)
    elif orientation_value == 8:
        image = image.rotate(90, expand=True)
    return image


flag

274

In [9]:
v

'Orientation'

In [10]:
img = load_image("dataset/images/val/ID_a7d9oI.jpeg")

In [11]:
# Path to the test images directory
test_dir_path = VAL_IMAGES_DIR

# Get a list of all image files in the test directory
image_files = os.listdir(test_dir_path)

# Initialize an empty list to store the results for all images
all_data = []

# Initialize an empty list to store the results for all images
all_data = []

# Iterate through each image in the directory
for image_file in tqdm(image_files):
	# Full path to the image
	img_path = os.path.join(test_dir_path, image_file)

	# Make predictions on the image
	results = model.predict(
		load_image(img_path),
		conf=0.0,
		imgsz=1024,
		max_det=30,
		verbose=False,
	)  # verbose=False,

	# Extract bounding boxes, confidence scores, and class labels
	boxes = (
		results[0].boxes.xyxy.tolist() if results[0].boxes else []
	)  # Bounding boxes in xyxy format
	classes = results[0].boxes.cls.tolist() if results[0].boxes else []  # Class indices
	confidences = (
		results[0].boxes.conf.tolist() if results[0].boxes else []
	)  # Confidence scores
	names = results[0].names  # Class names dictionary

	if boxes:  # If detections are found
		for box, cls, conf in zip(boxes, classes, confidences):
			x1, y1, x2, y2 = box
			detected_class = names[
				int(cls)
			]  # Get the class name from the names dictionary

			# Add the result to the all_data list
			all_data.append(
				{
					"Image_ID": str(image_file),
					"class": detected_class,
					"confidence": conf,
					"ymin": y1,
					"xmin": x1,
					"ymax": y2,
					"xmax": x2,
				}
			)
	else:  # If no objects are detected
		all_data.append(
			{
				"Image_ID": str(image_file),
				"class": "None",
				"confidence": None,
				"ymin": None,
				"xmin": None,
				"ymax": None,
				"xmax": None,
			}
		)

100%|██████████| 1106/1106 [02:25<00:00,  7.62it/s]


In [12]:
# Convert the list to a DataFrame for all images
sub = pd.DataFrame(all_data)

In [13]:
sub.head()

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
0,ID_MYSxE2.jpg,healthy,0.793660,155.002014,401.393097,3211.408936,1952.548584
1,ID_MYSxE2.jpg,healthy,0.186654,0.000000,1491.179932,526.956238,2448.000000
2,ID_MYSxE2.jpg,healthy,0.113677,0.279762,1260.710938,509.153992,1844.228638
3,ID_MYSxE2.jpg,healthy,0.073360,0.399897,1009.076050,518.043335,1877.374878
4,ID_MYSxE2.jpg,healthy,0.068918,0.592209,1285.380005,629.944092,2448.000000


In [14]:
sub.describe()

,confidence,ymin,xmin,ymax,xmax
count,33180.000000,33180.000000,33180.000000,33180.000000,33180.000000
mean,0.046388,620.954887,636.202771,1414.991900,1262.737648
std,0.150589,862.959475,820.936554,1150.264527,1027.264068
min,0.000077,0.000000,0.000000,7.137871,8.674245
25%,0.001378,0.398175,0.575666,458.326958,416.000000
50%,0.003444,200.412468,300.937759,1080.000000,959.889343
75%,0.010525,951.255646,913.313217,2048.000000,1841.905151
max,0.899976,3939.700439,3960.417725,4128.000000,4128.000000


In [15]:
sub['class'].value_counts()

class
cssvd          12239
healthy        11853
anthracnose     9088
Name: count, dtype: int64

In [16]:
def load_yolo_labels(label_folder):
	label_data = {}
	label_folder = Path(label_folder)

	for label_file in label_folder.glob("*"):
		with open(label_file, "r") as file:
			annotations = []
			for line in file:
				parts = line.strip().split()
				if len(parts) == 5:
					class_id, x_center, y_center, width, height = map(float, parts)
					annotations.append({
						"class_id": int(class_id),
						"x_center": x_center,
						"y_center": y_center,
						"width": width,
						"height": height
					})
				else:
					print(f"Skipping line in {label_file}: {line.strip()}")
			label_data[label_file.stem] = annotations
	# Convert the label data to a pandas DataFrame
	label_df = []
	for image_id, annotations in label_data.items():
		for annotation in annotations:
			label_df.append({
				"Image_ID": image_id,
				"class_id": annotation["class_id"],
				"x_center": annotation["x_center"],
				"y_center": annotation["y_center"],
				"width": annotation["width"],
				"height": annotation["height"]
			})

	label_df = pd.DataFrame(label_df)
	return label_df

# Example usage
label_folder = VAL_LABELS_DIR
labels = load_yolo_labels(label_folder)
labels.sample(5)

def yolo_to_bbox(image_folder, labels_df: pd.DataFrame):
	image_folder = Path(image_folder)
	converted_bboxes = []
	for image_file in image_folder.glob("*"):
		image_id = image_file.stem
		if image_id not in labels_df['Image_ID'].values:
			converted_bboxes.append({
				"Image_ID": image_id,
				"class_id": -1,  # Indicating no label
				"xmin": None,
				"ymin": None,
				"xmax": None,
				"ymax": None
			})

	for _, row in labels_df.iterrows():
		all_ids = list(image_folder.glob(f"{row['Image_ID']}.*"))
		image_path = image_folder / f"{row['Image_ID']}"
		if all_ids:
			image_path = all_ids[0]

		if image_path.exists():
			img = load_image(image_path)
			img_width, img_height = img.size

			x_center = row['x_center'] * img_width
			y_center = row['y_center'] * img_height
			width = row['width'] * img_width
			height = row['height'] * img_height

			x_min = x_center - (width / 2)
			y_min = y_center - (height / 2)
			x_max = x_center + (width / 2)
			y_max = y_center + (height / 2)

			converted_bboxes.append({
				"Image_ID": row['Image_ID'],
				"class_id": row['class_id'],
				"xmin": x_min,
				"ymin": y_min,
				"xmax": x_max,
				"ymax": y_max
			})
		else:
			print(f"Image {image_path} not found.")

	return pd.DataFrame(converted_bboxes)

# Example usage
converted_labels = yolo_to_bbox(VAL_IMAGES_DIR, labels)
converted_labels.sample(5)

,Image_ID,class_id,xmin,ymin,xmax,ymax
784,ID_dfJsQ9,2,459.000864,279.000288,1881.000576,3231.000864
1910,ID_IcRNaZ,1,0.000480,0.000000,645.000480,1280.000000
163,ID_A4KXll,2,0.000000,41.000320,898.999680,1280.000640
199,ID_tlhJYi,1,222.000000,52.999680,856.999680,1280.000000
1609,ID_dvs1xy,0,-0.001548,110.999856,1299.999564,2654.999568


In [17]:
converted_labels.describe()

,class_id,xmin,ymin,xmax,ymax
count,1962.000000,1962.000000,1962.000000,1962.000000,1962.000000
mean,1.222732,423.476536,449.040751,1449.574923,1795.911814
std,0.786055,589.985397,598.362691,1010.998380,1174.059673
min,0.000000,-0.002016,-0.002064,32.999824,23.999872
25%,1.000000,9.999884,28.999680,625.999086,860.000160
50%,1.000000,188.999856,195.999896,1209.500688,1524.499456
75%,2.000000,580.001508,632.750234,2217.750633,2786.751000
max,2.000000,3612.000672,3591.000000,4128.000000,4128.002064


In [18]:
converted_labels["Image_ID"].nunique()

1106

In [19]:
converted_labels['class_id'].value_counts()

class_id
2    873
1    653
0    436
Name: count, dtype: int64

In [20]:
class_map

{'anthracnose': 0, 'cssvd': 1, 'healthy': 2}

In [21]:
id_class_map = {v: k for k, v in class_map.items()}
converted_labels['class'] = converted_labels['class_id'].map(id_class_map)
converted_labels['class'].value_counts()

class
healthy        873
cssvd          653
anthracnose    436
Name: count, dtype: int64

In [22]:
converted_labels.sample(5)

,Image_ID,class_id,xmin,ymin,xmax,ymax,class
761,ID_mw6pUE,0,47.998944,937.000008,4031.997984,2854.001304,anthracnose
1587,ID_D8AD7q,2,122.999968,1.000064,373.000160,380.000192,healthy
1005,ID_OWQm05,1,388.999680,161.998848,1412.000256,1696.999424,cssvd
311,ID_hK8Sxt,1,0.000000,21.000320,742.999680,1222.000000,cssvd
651,ID_ET34jY,0,477.000000,888.000000,1800.000000,4000.000000,anthracnose


In [23]:
sub.sample(5)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
18968,ID_O4dd22.jpg,healthy,0.009728,3066.913086,996.172546,3264.000000,1642.625366
19038,ID_T3jhqx.jpg,cssvd,0.000654,389.841553,1460.761230,955.861328,1536.000000
20737,ID_tSdU7L.jpg,anthracnose,0.016608,2043.214600,2502.109619,4127.622559,3095.925781
29971,ID_Qr3Qae.jpg,healthy,0.583594,1316.103271,587.824219,1855.371704,926.790833
22127,ID_O4wYot.JPG,cssvd,0.006454,1044.017090,488.935913,2393.317139,767.194336


In [24]:
sub.loc[:, "Image_ID"] = sub["Image_ID"].apply(lambda x: str(Path(x).stem))

sub.sample(3)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
6505,ID_WB25bW,anthracnose,0.000362,274.145691,2236.656982,1790.266235,3988.510742
14483,ID_JqilcR,healthy,0.001162,2601.183838,608.301392,3024.000000,3286.479004
9166,ID_Ms0aHF,cssvd,0.003119,1755.865723,258.861450,2047.590332,530.046631


In [25]:
def convert_df(df: pd.DataFrame):
	df = df.copy().dropna()
	return {
		img_id: {
			"boxes": torch.tensor(raw[["xmin", "ymin", "xmax", "ymax"]].values, dtype=torch.float32),
			"scores": (
				torch.tensor(raw["confidence"].values, dtype=torch.float32)
				if "confidence" in raw.columns
				else None
			),
			"labels": torch.tensor(raw["class_id"].values, dtype=torch.int32),
		}
		for (img_id, ), raw in df.groupby(["Image_ID"])
	}

def default_value():
	return {
		"boxes": torch.empty((0, 4), dtype=torch.float32),
		"scores": torch.empty((0,), dtype=torch.float32),
		"labels": torch.empty((0,), dtype=torch.int32),
	}

def get_preds_data(preds, thr: float = 0.5):
	if thr is not None:
		preds = preds[preds["confidence"] >= thr]
	preds = convert_df(preds)
	d = default_value()
	return {i: preds.get(i, d) for i in converted_labels["Image_ID"].unique()}

In [26]:
converted_labels.isna().sum()

Image_ID    0
class_id    0
xmin        0
ymin        0
xmax        0
ymax        0
class       0
dtype: int64

In [27]:
ground_truth = convert_df(converted_labels)
ground_truth = {k: ground_truth[k] for k in converted_labels["Image_ID"].unique()}

len(ground_truth)

1106

In [28]:
ground_truth["ID_A4KXll"]

{'boxes': tensor([[   0.0000,   41.0003,  898.9997, 1280.0006]]),
 'scores': None,
 'labels': tensor([2], dtype=torch.int32)}

In [29]:
import torch

def calculate_iou_tensor(box1, box2):
	"""
	box1: [4], box2: [4]
	Format: [xmin, ymin, xmax, ymax]
	"""
	xA = torch.max(box1[0], box2[0])
	yA = torch.max(box1[1], box2[1])
	xB = torch.min(box1[2], box2[2])
	yB = torch.min(box1[3], box2[3])

	inter_area = torch.clamp(xB - xA, min=0) * torch.clamp(yB - yA, min=0)
	box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
	box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
	union_area = box1_area + box2_area - inter_area
	return inter_area / union_area if union_area > 0 else torch.tensor(0.0)

def evaluate_detection(predictions, ground_truths, iou_threshold=0.5, conf_threshold=0.0):
	"""
	predictions: list of dicts (len = batch size), each dict with 'boxes', 'scores', 'labels'
	ground_truths: list of dicts with 'boxes', 'labels'
	"""
	TP = 0
	FP = 0
	FN = 0

	for preds, gts in zip(predictions, ground_truths):
		pred_boxes = preds['boxes']
		pred_labels = preds['labels']
		pred_scores = preds['scores'] if preds['scores'] is not None else torch.ones(len(pred_boxes))

		gt_boxes = gts['boxes']
		gt_labels = gts['labels']
		matched_gt = set()

		for i in range(len(pred_boxes)):
			if pred_scores[i] < conf_threshold:
				continue
			pred_box = pred_boxes[i]
			pred_label = pred_labels[i]
			match_found = False

			for j in range(len(gt_boxes)):
				if j in matched_gt:
					continue
				if pred_label != gt_labels[j]:
					continue
				iou = calculate_iou_tensor(pred_box, gt_boxes[j])
				if iou >= iou_threshold:
					TP += 1
					matched_gt.add(j)
					match_found = True
					break
			if not match_found:
				FP += 1

		FN += len(gt_boxes) - len(matched_gt)

	precision = TP / (TP + FP) if (TP + FP) else 0.0
	recall = TP / (TP + FN) if (TP + FN) else 0.0
	f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) else 0.0
	accuracy = TP / (TP + FP + FN) if (TP + FP + FN) else 0.0

	return {
		'TP': TP,
		'FP': FP,
		'FN': FN,
		'Precision': precision,
		'Recall': recall,
		'F1 Score': f1_score,
		'Accuracy': accuracy
	}

In [30]:
sub.sample(3)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
24032,ID_JRxgFk,cssvd,0.068093,469.527466,0.842497,1709.327637,895.773560
31355,ID_Uisb4e,cssvd,0.008285,35.990952,110.033119,1006.502747,382.981537
11443,ID_hhZ51g,healthy,0.002120,337.367188,1195.939453,576.000000,1280.000000


In [31]:
sub["class_id"] = sub["class"].map(class_map)

sub.sample(3)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
23211,ID_D8AD7q,healthy,0.002125,0.000000,271.965302,20.165335,401.354034,2
6995,ID_CbluH3,cssvd,0.018356,524.520935,118.444290,805.456482,909.433899,1
3599,ID_v0zx0A,cssvd,0.001343,9.082642,0.000000,757.140503,755.332153,1


In [32]:
predictions = get_preds_data(sub, None)

len(predictions)

1106

In [33]:
for i in np.linspace(0.0, 0.95, 15):
	scores = evaluate_detection(
		predictions.values(),
		ground_truth.values(),
		iou_threshold=0.5,
		conf_threshold=i
	)
	print("Evaluation metric at:", i, " score :", scores)

Evaluation metric at: 0.0  score : {'TP': 1791, 'FP': 31389, 'FN': 171, 'Precision': 0.05397830018083183, 'Recall': 0.9128440366972477, 'F1 Score': 0.10192931534915486, 'Accuracy': 0.053701538184762074}
Evaluation metric at: 0.06785714285714285  score : {'TP': 1605, 'FP': 1510, 'FN': 357, 'Precision': 0.5152487961476726, 'Recall': 0.8180428134556575, 'F1 Score': 0.6322631475280679, 'Accuracy': 0.46226958525345624}
Evaluation metric at: 0.1357142857142857  score : {'TP': 1523, 'FP': 875, 'FN': 439, 'Precision': 0.6351125938281902, 'Recall': 0.7762487257900101, 'F1 Score': 0.6986238532110093, 'Accuracy': 0.5368346845259077}
Evaluation metric at: 0.20357142857142857  score : {'TP': 1451, 'FP': 602, 'FN': 511, 'Precision': 0.7067705796395519, 'Recall': 0.7395514780835881, 'F1 Score': 0.7227895392278956, 'Accuracy': 0.5659126365054602}
Evaluation metric at: 0.2714285714285714  score : {'TP': 1397, 'FP': 436, 'FN': 565, 'Precision': 0.762138570649209, 'Recall': 0.7120285423037717, 'F1 Score'

In [34]:
converted_labels.sample(5)

,Image_ID,class_id,xmin,ymin,xmax,ymax,class
19,ID_rAK95H,1,161.000160,0.000000,582.999840,310.00064,cssvd
1119,ID_Ss1mkM,2,114.000224,15.000128,333.000096,403.00000,healthy
1324,ID_ueaZEa,1,7.999360,234.999840,222.999680,405.99984,cssvd
542,ID_kI532t,1,65.999360,41.999520,1280.000000,933.99984,cssvd
786,ID_mUk7Wy,1,53.000448,76.000256,1536.000768,1936.00000,cssvd


In [35]:
import torch
from torchmetrics.detection import MeanAveragePrecision

def compute_map(preds, targets, iou_thresholds):
	"""
	Compute mAP at different IoU thresholds using torchmetrics.
	
	Args:
		preds: List of dicts with 'boxes', 'scores', 'labels' for predictions
		targets: List of dicts with 'boxes', 'labels' for ground truth
		iou_thresholds: List of IoU thresholds to evaluate
	
	Returns:
		Dict containing mAP results for each IoU threshold
	"""
	# Initialize the metric
	metric = MeanAveragePrecision(iou_thresholds=iou_thresholds)
	
	# Update metric with predictions and targets
	metric.update(preds, targets)
	
	# Compute the results
	result = metric.compute()
	
	return result

thrs = np.linspace(0.0, 0.95, 15)
# Example usage
# if __name__ == "__main__":
# Example predictions and targets
iou_thresholds = [0.5]
for i in thrs:
	preds = list(get_preds_data(sub, i).values())

	targets = list(ground_truth.values())

	# Compute mAP
	results = compute_map(preds, targets, iou_thresholds)

	# Print results
	print("mAP Results:", i, " - ", results)

mAP Results: 0.0  -  {'map': tensor(0.7602), 'map_50': tensor(0.7602), 'map_75': tensor(-1.), 'map_small': tensor(0.), 'map_medium': tensor(0.5499), 'map_large': tensor(0.7644), 'mar_1': tensor(0.5209), 'mar_10': tensor(0.8840), 'mar_100': tensor(0.9129), 'mar_small': tensor(0.), 'mar_medium': tensor(0.8298), 'mar_large': tensor(0.9153), 'map_per_class': tensor(-1.), 'mar_100_per_class': tensor(-1.), 'classes': tensor([0, 1, 2], dtype=torch.int32)}
mAP Results: 0.06785714285714285  -  {'map': tensor(0.7326), 'map_50': tensor(0.7326), 'map_75': tensor(-1.), 'map_small': tensor(0.), 'map_medium': tensor(0.5116), 'map_large': tensor(0.7383), 'mar_1': tensor(0.5186), 'mar_10': tensor(0.8166), 'mar_100': tensor(0.8173), 'mar_small': tensor(0.), 'mar_medium': tensor(0.6170), 'mar_large': tensor(0.8221), 'map_per_class': tensor(-1.), 'mar_100_per_class': tensor(-1.), 'classes': tensor([0, 1, 2], dtype=torch.int32)}
mAP Results: 0.1357142857142857  -  {'map': tensor(0.7093), 'map_50': tensor(0

In [36]:
sub.to_csv('dataset/evaluations/validation.csv', index=False)

In [37]:
import pandas as pd

sub = pd.read_csv('dataset/evaluations/validation.csv')
sub.sample(5)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
32979,ID_auKPn1,cssvd,0.007646,0.025009,0.000000,481.408936,219.758621,1
25821,ID_MhaDzk,cssvd,0.002620,0.456421,433.101807,160.686188,989.409790,1
1869,ID_qjhrcu,healthy,0.001687,1409.287476,0.000000,3011.021729,615.813660,2
14830,ID_HTXcz3,healthy,0.005061,14.592889,236.308640,92.395645,391.573212,2
24229,ID_gJfSc6,healthy,0.042228,0.014564,429.296936,133.343689,790.732666,2
